Following http://nbviewer.jupyter.org/github/blei-lab/edward/blob/master/notebooks/supervised_regression.ipynb

### Setup system

In [1]:
import pandas     as pd
import tensorflow as tf
import edward     as ed
import numpy      as np

from edward.models import Normal

### Access the data

In [2]:
df_data = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', index_col=0)

In [3]:
df_data.shape

(200, 4)

In [4]:
df_data.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


Convert pandas data frame to numpy array.

In [5]:
data = df_data.values

Define training and test data sets.

In [6]:
X_train = df_data[['TV', 'Radio', 'Newspaper']].iloc[:100].values
X_test  = df_data[['TV', 'Radio', 'Newspaper']].iloc[100:200].values
y_train = df_data['Sales'].iloc[:100].values
y_test  = df_data['Sales'].iloc[100:200].values

N = 100  # number of data points in each data set
D = 3    # number of features

### Specify model

In [7]:
X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

### Fit model

In [8]:
qw = Normal(loc=tf.Variable(tf.random_normal([D])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
qb = Normal(loc=tf.Variable(tf.random_normal([1])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [9]:
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(n_samples=5, n_iter=250)

250/250 [100%] ██████████████████████████████ Elapsed: 5s | Loss: 7489.408


### Evaluate model

First, compute the posterior predictive distribution.

In [10]:
y_posterior = Normal(loc=ed.dot(X, qw) + qb, scale=tf.ones(N))

Next, compute the mean squared and absolute error of the prediction on the test data set.

In [11]:
ed.evaluate('mean_squared_error', data={X: X_test, y_posterior: y_test})

6.9637346

In [12]:
ed.evaluate('mean_absolute_error', data={X: X_test, y_posterior: y_test})

2.3549125